# AutoRes Evaluator

<a href="https://colab.research.google.com/github/fuyu-quant/autores-evaluator/blob/main/example/logistc_regression.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install autoresevaluator

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

from autoresevaluator import AutoResEvaluator

## サンプルのモデルを作成

In [11]:
# 工夫のないロジスティック回帰
sample_model = """
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, weights):
    m = len(y)
    h = sigmoid(X.dot(weights))
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost

def gradient_descent(X, y, weights, learning_rate, iterations):
    m = len(y)
    cost_history = []

    for i in range(iterations):
        h = sigmoid(X.dot(weights))
        gradient = np.dot(X.T, (h - y)) / m
        weights -= learning_rate * gradient
        cost = compute_cost(X, y, weights)
        cost_history.append(cost)

    return weights, cost_history

def predict(X, weights):
    predictions = sigmoid(X.dot(weights)) >= 0.5
    return predictions.astype(int)

def model(X_train, y_train, X_valid, params):
    # 訓練データにバイアス項を追加
    X_train = np.insert(X_train, 0, 1, axis=1)
    X_valid = np.insert(X_valid, 0, 1, axis=1)

    # 重みの初期化
    weights = np.zeros(X_train.shape[1])

    # モデルの学習
    weights, cost_history = gradient_descent(X_train, y_train, weights, params["learning_rate"], params["iterations"])

    # 検証データセットに対する予測
    y_pred = predict(X_valid, weights)

    return y_pred
"""


with open('/content/example.py', 'w') as file:
    file.write(sample_model)

In [7]:
# いくつか追加したロジスティック回帰
sample_model = """
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def feature_normalize(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

def compute_cost_reg(X, y, weights, lambda_):
    m = len(y)
    h = sigmoid(X @ weights)
    cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) + (lambda_ / (2*m)) * np.sum(weights[1:] ** 2)
    return cost

def gradient_descent_reg(X, y, weights, learning_rate, iterations, lambda_):
    m = len(y)
    cost_history = []
    for i in range(iterations):
        h = sigmoid(X @ weights)
        gradient = (X.T @ (h - y)) / m
        gradient[1:] += (lambda_ / m) * weights[1:]  # バイアス項は正則化しない
        weights -= learning_rate * gradient
        cost = compute_cost_reg(X, y, weights, lambda_)
        cost_history.append(cost)

        # 早期停止の条件（改善が見られない場合）
        if i > 0 and abs(cost_history[-2] - cost_history[-1]) < 1e-6:
            print(f"Early stopping at iteration {i}")
            break

    return weights, cost_history

def predict(X, weights):
    predictions = sigmoid(X @ weights) >= 0.5
    return predictions.astype(int)

def model(X_train, y_train, X_valid, params):
    # 特徴量の正規化
    X_train_norm, mu, sigma = feature_normalize(X_train)
    X_valid_norm = (X_valid - mu) / sigma  # 検証データも同じパラメータで正規化

    # 訓練データにバイアス項を追加
    X_train_norm = np.insert(X_train_norm, 0, 1, axis=1)
    X_valid_norm = np.insert(X_valid_norm, 0, 1, axis=1)

    # 重みの初期化
    weights = np.zeros(X_train_norm.shape[1])

    # モデルの学習
    lambda_ = params["lambda"]  # 正則化パラメータ
    weights, cost_history = gradient_descent_reg(X_train_norm, y_train, weights, params["learning_rate"], params["iterations"], lambda_)

    # 検証データセットに対する予測
    y_pred = predict(X_valid_norm, weights)

    return y_pred
"""


with open('/content/example.py', 'w') as file:
    file.write(sample_model)

## AutoResEvaluatorの設定

In [12]:
params = {
    'learning_rate': {'type': 'log_float', 'args': [1e-8, 0.1]},
    'iterations': {'type': 'log_float', 'args': [100, 1000]},
    "lambda": 1
}

In [13]:
are = AutoResEvaluator(
    task_type='tabledata binary classification',
    dataset_name='titanic',
    model_path='/content/example.py',
    params=params,
    valuation_index='roc_auc'
    )

## AutoResEvaluatorの実行

In [14]:
are.exec()

INFO:ResultLogger:------AutoRes Evaluator Start------
INFO:ResultLogger:task type: tabledata binary classification
INFO:ResultLogger:dataset name: titanic
INFO:ResultLogger:model path: /content/example.py
INFO:ResultLogger:valuation_index: roc_auc
INFO:ResultLogger:objective: maximize
INFO:ResultLogger:------Optuna start------


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:ResultLogger:------Cross validataion------
INFO:ResultLogger:------Round1------
INFO:ModelErrorLogger:ModuleSpec: ModuleSpec(name='copy_file', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7813361caaa0>, origin='/content/copy_file.py')
INFO:ModelErrorLogger:Module: <module 'copy_file' from '/content/copy_file.py'>
INFO:ModelErrorLogger:メソッドが正常にインポートされました
ERROR:ModelErrorLogger:Exec Error: 'float' object cannot be interpreted as an integer
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autoresevaluator/train/exec_model.py", line 23, in exec_model
    y_pred = model(X_train, y_train, X_test, params)
  File "/content/copy_file.py", line 39, in model
    weights, cost_history = gradient_descent(X_train, y_train, weights, params["learning_rate"], params["iterations"])
  File "/content/copy_file.py", line 17, in gradient_descent
    for i in range(iterations):
TypeError: 'float' object cannot be interpreted as an integer
INFO:ModelEr